# 06-04

- KNN Clustering 
- Using Categories from KNN to get subset from information
- Continous Features(Transition and Highlevel features)
    - MinMax Scaling
    - Remove Constant Features
- Categorical Features
- Gower


In [49]:
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt 
import math
from ipynb.fs.full.Utils import createKDeplot, loadDataFrames, dropFeaturesWithNoVariance, loadCoverGroups
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import stats
from ipynb.fs.full.Pipeline import removeConstantFeatures, applyBoxCoxAllFeatures, centerAndScale, applyEuclideanDistance, getIndexAndVector, applyBoxCoxFeatures,applyJaccard, getEuclideanJaccardDf, scalerMinMax, applyGower
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [50]:
transitionMatrices = pd.read_csv('./data/unified/07-transitionsClustered.csv')
transitionMatrices = transitionMatrices[['id','category']]

In [51]:
cover = loadCoverGroups()
listCovers = cover[20]
originalSongId = listCovers[0]
coverSongId = listCovers[1]

In [52]:
coverSongId

6000670

In [53]:
df = pd.read_csv("./data/unified/05-allDataContinousCategoricalTransition.csv")
df = df.drop(columns=['Unnamed: 0', 'index'])

In [54]:
transitionMatrices

,id,category
0,23,1
1,105,0
2,118,2
3,120,1
4,292,2
...,...,...
5975,6002496,0
5976,6002497,1
5977,6002498,1
5978,6002499,1


In [55]:
df = pd.merge(df,transitionMatrices, on='id')

In [56]:
categoryOriginalSong = df[df.id == originalSongId].category.values[0]
df = df[df.category == categoryOriginalSong]

In [57]:
df

,id,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,...,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2,category
2,118,0.614,0.755,0.0276,0.01620,0.0831,0.6450,92.622,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2
4,292,0.496,0.527,0.0339,0.19500,0.1380,0.2830,155.932,0.013889,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2
6,418,0.519,0.812,0.0278,0.00174,0.1540,0.6490,141.872,0.038095,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2
7,456,0.457,0.488,0.0301,0.32600,0.1540,0.2830,72.517,0.025641,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2
8,467,0.533,0.512,0.0288,0.04750,0.0442,0.5530,75.984,0.028571,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5913,6002435,0.676,0.572,0.0405,0.04490,0.0979,0.0398,149.997,0.070423,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2
5926,6002448,0.726,0.837,0.0320,0.25700,0.3220,0.5590,120.040,0.010000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2
5945,6002467,0.845,0.938,0.0502,0.00306,0.0896,0.7550,127.007,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,2
5962,6002486,0.548,0.542,0.1390,0.27400,0.0967,0.6490,174.043,0.000000,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2


In [58]:
df = df.drop(columns=['category'])

In [59]:
df

,id,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
2,118,0.614,0.755,0.0276,0.01620,0.0831,0.6450,92.622,0.000000,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,292,0.496,0.527,0.0339,0.19500,0.1380,0.2830,155.932,0.013889,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,418,0.519,0.812,0.0278,0.00174,0.1540,0.6490,141.872,0.038095,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,456,0.457,0.488,0.0301,0.32600,0.1540,0.2830,72.517,0.025641,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,467,0.533,0.512,0.0288,0.04750,0.0442,0.5530,75.984,0.028571,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5913,6002435,0.676,0.572,0.0405,0.04490,0.0979,0.0398,149.997,0.070423,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5926,6002448,0.726,0.837,0.0320,0.25700,0.3220,0.5590,120.040,0.010000,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5945,6002467,0.845,0.938,0.0502,0.00306,0.0896,0.7550,127.007,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
5962,6002486,0.548,0.542,0.1390,0.27400,0.0967,0.6490,174.043,0.000000,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [60]:
## Removing unamed and other unnecessary columns
df.reset_index(drop=True, inplace=True)
df = df.sort_values('id')
df.id = df.id.astype('int64')
# Saving idChords
ids = df.id
df = df.set_index('id')
df.reset_index(drop=True, inplace=True)

In [61]:
df

,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,2,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
0,0.614,0.755,0.0276,0.01620,0.0831,0.6450,92.622,0.000000,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.496,0.527,0.0339,0.19500,0.1380,0.2830,155.932,0.013889,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.519,0.812,0.0278,0.00174,0.1540,0.6490,141.872,0.038095,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.457,0.488,0.0301,0.32600,0.1540,0.2830,72.517,0.025641,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.533,0.512,0.0288,0.04750,0.0442,0.5530,75.984,0.028571,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,0.676,0.572,0.0405,0.04490,0.0979,0.0398,149.997,0.070423,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1378,0.726,0.837,0.0320,0.25700,0.3220,0.5590,120.040,0.010000,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1379,0.845,0.938,0.0502,0.00306,0.0896,0.7550,127.007,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1380,0.548,0.542,0.1390,0.27400,0.0967,0.6490,174.043,0.000000,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [62]:
ids[ids ==originalSongId]

1158    6000771
Name: id, dtype: int64

In [63]:
df

,danceability,energy,speechiness,acousticness,liveness,valence,tempo,0,1,2,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
0,0.614,0.755,0.0276,0.01620,0.0831,0.6450,92.622,0.000000,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.496,0.527,0.0339,0.19500,0.1380,0.2830,155.932,0.013889,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.519,0.812,0.0278,0.00174,0.1540,0.6490,141.872,0.038095,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.457,0.488,0.0301,0.32600,0.1540,0.2830,72.517,0.025641,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.533,0.512,0.0288,0.04750,0.0442,0.5530,75.984,0.028571,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,0.676,0.572,0.0405,0.04490,0.0979,0.0398,149.997,0.070423,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1378,0.726,0.837,0.0320,0.25700,0.3220,0.5590,120.040,0.010000,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1379,0.845,0.938,0.0502,0.00306,0.0896,0.7550,127.007,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1380,0.548,0.542,0.1390,0.27400,0.0967,0.6490,174.043,0.000000,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## Pipeline

In [64]:
dfEuclidean, dfJaccard = getEuclideanJaccardDf(df)

In [65]:
dfEuclidean = removeConstantFeatures(dfEuclidean)
dfEuclidean = scalerMinMax(dfEuclidean)

In [66]:
dfEuclidean

,0,1,2,3,4,5,6,7,8,9,...,649,650,651,652,653,654,655,656,657,658
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.044476,0.016252,0.609321,0.763720,0.051802,0.011578,0.214746,0.646309
1,0.019444,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.195773,0.474526,0.532511,0.109138,0.025880,0.635374,0.261816
2,0.053333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.001734,0.500800,0.821523,0.125849,0.012032,0.541960,0.650558
3,0.035897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.327300,0.429975,0.492962,0.125849,0.017253,0.081169,0.261816
4,0.040000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.047678,0.516792,0.517300,0.011175,0.014302,0.104204,0.548593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,0.098592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.045068,0.680146,0.578145,0.067258,0.040863,0.595942,0.003505
1378,0.014000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.258022,0.737263,0.846875,0.301305,0.021566,0.396909,0.554965
1379,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.003059,0.873201,0.949296,0.058590,0.062883,0.443198,0.763144
1380,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.275091,0.533927,0.547722,0.066005,0.264472,0.755702,0.650558


In [67]:
dfJaccard

,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,...,mood_5,mood_6,mood_7,mood_8,mood_9,mood_10,mood_11,instrumentalness_cat_0,instrumentalness_cat_1,instrumentalness_cat_2
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [68]:
df = pd.merge(dfJaccard,dfEuclidean, left_index=True, right_index=True)

In [69]:
df

,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,...,649,650,651,652,653,654,655,656,657,658
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.044476,0.016252,0.609321,0.763720,0.051802,0.011578,0.214746,0.646309
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.195773,0.474526,0.532511,0.109138,0.025880,0.635374,0.261816
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.001734,0.500800,0.821523,0.125849,0.012032,0.541960,0.650558
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.000000,0.327300,0.429975,0.492962,0.125849,0.017253,0.081169,0.261816
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.047678,0.516792,0.517300,0.011175,0.014302,0.104204,0.548593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.000000,0.045068,0.680146,0.578145,0.067258,0.040863,0.595942,0.003505
1378,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.000000,0.258022,0.737263,0.846875,0.301305,0.021566,0.396909,0.554965
1379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.000000,0.003059,0.873201,0.949296,0.058590,0.062883,0.443198,0.763144
1380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.275091,0.533927,0.547722,0.066005,0.264472,0.755702,0.650558


In [70]:
ranksDFGower = applyGower(df,ids, originalSongId,len(df))
ranksDFGower.head(10)

,id,values
0,6000771,0.000000
1,924061,0.013700
2,40013,0.015760
3,6000670,0.016257
4,118655,0.017571
5,2628012,0.017824
6,1709964,0.018146
7,582263,0.018240
8,695153,0.018393
9,1243779,0.018433


In [71]:
ranksDFGower.loc[ranksDFGower.id.isin(listCovers)]

,id,values
0,6000771,0.000000
3,6000670,0.016257


In [72]:
ranksDFGower[ranksDFGower['id']==coverSongId]

,id,values
3,6000670,0.016257
